## Import packages

In [205]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Pre-define some colors

In [206]:
blue = (255, 0, 0)
green = (0, 255, 0)
red = (0, 0, 255)

## Helper functions

In [287]:
def show_image(img):
    cv2.imshow('image', img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 



def get_avg_circle(circles):
    avg = np.average(circles[0], axis=0)
    centerX, centerY, radius = avg
    return int(centerX), int(centerY), int(radius)



def calculate_distance_between_two_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)


# Get the distance between point p0 and a line p1 p2
# Use the formula: Line defined by two points (https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line)
def calcualte_distance_between_point_and_line(p1, p2, p0):
    return np.abs(np.cross(p2-p1, p1-p0)) / calculate_distance_between_two_points(p1, p2)




    



## Load gauge images

In [289]:
image = cv2.imread('Images/gauge-1.jpg')
height, width = image.shape[0:2]
grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

print(f"The size of the image: Height: [{height}] px, Width: [{width}] px")

show_image(image)

The size of the image: Height: [2199] px, Width: [1125] px


## Detect the circle of the dial

In [290]:
detectCircleResult = image.copy()

# dp=1 The accumulator matrix has the same size as the image
# minDist: The minimum distance between two detected circles
# param1: Upper threshold for the internal Canny edge detector
# param2: The accumulator threshold for the circle centers at the detection stage
circles = cv2.HoughCircles(grayImage, cv2.HOUGH_GRADIENT, dp=1, minDist=int(height/2), param1=100, param2=100, minRadius=int(width*0.2), maxRadius=int(height))

print(circles)
if circles is None:
    raise Exception("Cannot find any circles")


centerX, centerY, radius = get_avg_circle(circles)

# Draw the detected circle
detectCircleResult = cv2.circle(detectCircleResult, (int(centerX), int(centerY)), int(radius), green, 2, cv2.LINE_AA)  
# Draw the center
detectCircleResult = cv2.circle(detectCircleResult, (int(centerX), int(centerY)), 2, green, 1, cv2.LINE_AA)  

print(f"Detected circle: CenterX=[{centerX}], CenterY=[{centerY}], radius=[{radius}]")

show_image(detectCircleResult)


[[[553.5 898.5 501. ]]]
Detected circle: CenterX=[553], CenterY=[898], radius=[501]


## Crop the image

In [291]:
padding = -10
upperLeft = (centerX-(radius+padding), centerY-(radius+padding))
lowerRight = (centerX+(radius+padding), centerY+(radius+padding))


dialImage = image[upperLeft[1] : (lowerRight[1]+1), upperLeft[0] : (lowerRight[0]+1), :]

dialHeight, dialWidth = image.shape[0:2]

show_image(dialImage)

## Detect the line of the pointer

In [297]:
detectLineResult = dialImage.copy()

dialGrayImage = cv2.cvtColor(dialImage, cv2.COLOR_BGR2GRAY)
height, width = dialGrayImage.shape
show_image(dialGrayImage)

# dialGrayImage = cv2.medianBlur(dialGrayImage, 9)
# dialGrayImage = cv2.GaussianBlur(dialGrayImage, (5, 5), 0)
# show_image(dialGrayImage)

_, thresholdResultmage = cv2.threshold(dialGrayImage, thresh=150, maxval=255, type=cv2.THRESH_BINARY_INV)

# cannyEdgeDetectionResult = cv2.Canny(dialGrayImage, threshold1=50, threshold2=200, apertureSize=3, L2gradient=True)

show_image(thresholdResultmage)

# rho: Distance resolution of the accumulator in pixels
# theta: Angle resolution of the accumulator in radians
# threshold: Accumulator threshold parameter
# minLineLength: Minimum line length
# maxLineGap: Maximum allowed gap between points on the same line
lines = cv2.HoughLinesP(thresholdResultmage, rho=1, theta=np.pi/180, threshold=100, minLineLength=width//4, maxLineGap=0)

# print(lines)
if lines is None:
    raise Exception("Cannot find any lines")


validLines = []
distanceThreshold = 1
for l in lines:
    aLine = l[0]
    p1 = np.array(aLine[:2])
    p2 = np.array(aLine[2:])
    p0 = np.array([width//2, height//2])
    distance = calcualte_distance_between_point_and_line(p1, p2, p0)

    if distance <= distanceThreshold:
        validTwoPoints = np.concatenate((p1, p2))
        validLines.append(validTwoPoints)

print(validLines)

  





for l in validLines:
    aLine = l
    cv2.line(detectLineResult, (aLine[0], aLine[1]), (aLine[2], aLine[3]), green, 1, cv2.LINE_AA)

show_image(detectLineResult)

[array([220, 294, 513, 507], dtype=int32)]


In [ ]:
validLine = validLines[0]

x1, y1, x2, y2 = validLine
p1 = np.array([x1, y1])
p2 = np.array([x2, y2])

centerX = width//2
centerY = height//2
p0 = np.array([centerX, centerY])

distanceP1 = calculate_distance_between_two_points(p1, p0)
distanceP2 = calculate_distance_between_two_points(p2, p0)

p = p1
if distanceP2 > distanceP1:
    p = p2


yDistanceToCenter = np.abs(p[1] - p0[1])

if yDistanceToCenter == 0 and p[0] <= p0[0]:
    print(f"Angle: 270 degree")
elif yDistanceToCenter == 0 and p[0] >= p0[0]:
    print(f"Angle: 90 degree")
else:
    distanceToBaseLine = calcualte_distance_between_point_and_line(p0, np.array([centerX, 0]))
    





In [67]:
detectLineResult = image.copy()

# threshold, result = cv2.threshold(grayImage, 200, 255, cv2.THRESH_BINARY_INV)

# show_image(result)


cannyEdgeDetectionResult = cv2.Canny(grayImage, threshold1=50, threshold2=200, apertureSize=3)

show_image(cannyEdgeDetectionResult)

# rho: Distance resolution of the accumulator in pixels
# theta: Angle resolution of the accumulator in radians
# threshold: Accumulator threshold parameter
# minLineLength: Minimum line length
# maxLineGap: Maximum allowed gap between points on the same line
lines = cv2.HoughLinesP(cannyEdgeDetectionResult, rho=1, theta=np.pi/180, threshold=100, minLineLength=50, maxLineGap=5)

print(lines)

if lines is not None:
    for l in lines:
        aLine = l[0]
        cv2.line(detectLineResult, (aLine[0], aLine[1]), (aLine[2], aLine[3]), green, 1, cv2.LINE_AA)

show_image(detectLineResult)

[[[ 57 149 112 145]]]
